In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.model_selection import KFold
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from ngboost import NGBRegressor
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
train = pd.read_csv('.\\data\\FIFA_train.csv')
test = pd.read_csv('.\\data\\FIFA_test.csv')
submission = pd.read_csv('.\\data\\submission.csv')

In [3]:
train.head()

,id,name,age,continent,contract_until,position,prefer_foot,reputation,stat_overall,stat_potential,stat_skill_moves,value
0,0,L. Messi,31,south america,2021,ST,left,5.0,94,94,4.0,110500000.0
1,3,De Gea,27,europe,2020,GK,right,4.0,91,93,1.0,72000000.0
2,7,L. Suárez,31,south america,2021,ST,right,5.0,91,91,3.0,80000000.0
3,8,Sergio Ramos,32,europe,2020,DF,right,4.0,91,91,3.0,51000000.0
4,9,J. Oblak,25,europe,2021,GK,right,3.0,90,93,1.0,68000000.0


In [4]:
train_data = train.drop(['value'],axis=1)

In [5]:
train_data

,id,name,age,continent,contract_until,position,prefer_foot,reputation,stat_overall,stat_potential,stat_skill_moves
0,0,L. Messi,31,south america,2021,ST,left,5.0,94,94,4.0
1,3,De Gea,27,europe,2020,GK,right,4.0,91,93,1.0
2,7,L. Suárez,31,south america,2021,ST,right,5.0,91,91,3.0
3,8,Sergio Ramos,32,europe,2020,DF,right,4.0,91,91,3.0
4,9,J. Oblak,25,europe,2021,GK,right,3.0,90,93,1.0
...,...,...,...,...,...,...,...,...,...,...,...
8927,16925,S. Adewusi,18,africa,2019,MF,right,1.0,48,63,3.0
8928,16936,C. Ehlich,19,europe,2020,DF,right,1.0,47,59,2.0
8929,16941,N. Fuentes,18,south america,2021,DF,right,1.0,47,64,2.0
8930,16942,J. Milli,18,europe,2021,GK,right,1.0,47,65,1.0


In [7]:
x_data=train[['stat_overall','reputation','stat_potential']]
value = train[['value']]
x_data.head()

,stat_overall,reputation,stat_potential
0,94,5.0,94
1,91,4.0,93
2,91,5.0,91
3,91,4.0,91
4,90,3.0,93


In [8]:
x_data1 = sm.add_constant(x_data, has_constant='add')

In [9]:
multi_model = sm.OLS(value, x_data)
fitted_multi_model = multi_model.fit()

In [10]:
fitted_multi_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  value   R-squared (uncentered):                   0.481
Model:                            OLS   Adj. R-squared (uncentered):              0.481
Method:                 Least Squares   F-statistic:                              2761.
Date:                Sat, 09 Jul 2022   Prob (F-statistic):                        0.00
Time:                        17:59:15   Log-Likelihood:                     -1.4982e+05
No. Observations:                8932   AIC:                                  2.996e+05
Df Residuals:                    8929   BIC:                                  2.997e+05
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
stat_overall    4.799e+04   9987.007      4.805      0.000    2.84e+04    6.76e+04
reputation      8.055e+06   1.32e+05     61.182      0.000     7.8e+06    8.31e+06
stat_potential -1.294e+05   8870.882    -14.592      0.000   -1.47e+05   -1.12e+05
==============================================================================
Omnibus:                     8107.625   Durbin-Watson:                   0.779
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           519774.397
Skew:                           4.148   Prob(JB):                         0.00
Kurtosis:                      39.439   Cond. No.                         264.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [11]:
train

,id,name,age,continent,contract_until,position,prefer_foot,reputation,stat_overall,stat_potential,stat_skill_moves,value
0,0,L. Messi,31,south america,2021,ST,left,5.0,94,94,4.0,110500000.0
1,3,De Gea,27,europe,2020,GK,right,4.0,91,93,1.0,72000000.0
2,7,L. Suárez,31,south america,2021,ST,right,5.0,91,91,3.0,80000000.0
3,8,Sergio Ramos,32,europe,2020,DF,right,4.0,91,91,3.0,51000000.0
4,9,J. Oblak,25,europe,2021,GK,right,3.0,90,93,1.0,68000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
8927,16925,S. Adewusi,18,africa,2019,MF,right,1.0,48,63,3.0,60000.0
8928,16936,C. Ehlich,19,europe,2020,DF,right,1.0,47,59,2.0,40000.0
8929,16941,N. Fuentes,18,south america,2021,DF,right,1.0,47,64,2.0,50000.0
8930,16942,J. Milli,18,europe,2021,GK,right,1.0,47,65,1.0,50000.0


In [19]:
x_data2 = train[['age','reputation','stat_overall','stat_potential','stat_skill_moves']]
x_data2.head()

,age,reputation,stat_overall,stat_potential,stat_skill_moves
0,31,5.0,94,94,4.0
1,27,4.0,91,93,1.0
2,31,5.0,91,91,3.0
3,32,4.0,91,91,3.0
4,25,3.0,90,93,1.0


In [20]:
x_data2_ = sm.add_constant(x_data2, has_constant='add')

In [21]:
multi_model2 = sm.OLS(value, x_data2_)
fitted_multi_model2 = multi_model.fit()

In [22]:
fitted_multi_model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  value   R-squared (uncentered):                   0.481
Model:                            OLS   Adj. R-squared (uncentered):              0.481
Method:                 Least Squares   F-statistic:                              2761.
Date:                Sat, 09 Jul 2022   Prob (F-statistic):                        0.00
Time:                        18:04:18   Log-Likelihood:                     -1.4982e+05
No. Observations:                8932   AIC:                                  2.996e+05
Df Residuals:                    8929   BIC:                                  2.997e+05
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
stat_overall    4.799e+04   9987.007      4.805      0.000    2.84e+04    6.76e+04
reputation      8.055e+06   1.32e+05     61.182      0.000     7.8e+06    8.31e+06
stat_potential -1.294e+05   8870.882    -14.592      0.000   -1.47e+05   -1.12e+05
==============================================================================
Omnibus:                     8107.625   Durbin-Watson:                   0.779
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           519774.397
Skew:                           4.148   Prob(JB):                         0.00
Kurtosis:                      39.439   Cond. No.                         264.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [23]:
fitted_multi_model.params

stat_overall      4.798709e+04
reputation        8.054977e+06
stat_potential   -1.294475e+05
dtype: float64